## Notes

This tests a Runga-Kutta-Nystrom method applied to the spherical active particle model

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def w(y,N=10,U=10.0):
    M = 1.0
    W = 1.0-y[1]**2
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += temp*np.cosh(kn*y[0])*np.cos(kn*y[1])
        M += temp
    return W*U/M
def dwdx(y,N=10,U=10.0):
    M = 1.0
    W = 0*y[0] # ensure correct shape...
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += kn*temp*np.sinh(kn*y[0])*np.cos(kn*y[1])
        M += temp
    return W*U/M
def dwdy(y,N=10,U=10.0):
    M = 1.0
    W = -2*y[1]
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += -kn*temp*np.cosh(kn*y[0])*np.sin(kn*y[1])
        M += temp
    return W*U/M

# Here I frame the system as a Hamiltonian and then solve using a (nearly?) symplectic 
# Runge-Kutta-Nystrom solver (a supposed 4th order method from wiki...)
def RKN_method(ode_fun,y0,dy0,dt,nt,t0=0.,args={}):
    """Here ode_fun is such that \ddot{y}=f(y)"""
    r3 = 3.0**0.5
    c1 = (3.+r3)/6.
    c2 = (3.-r3)/6.
    c3 = c1
    a21 = (2.-r3)/12.
    a32 = r3/6.
    bb1 = (5.-3.*r3)/24.
    bb2 = (3.+r3)/12.
    bb3 = (1.+r3)/24.
    b1 = (3.-2.*r3)/12.
    b2 = 0.5
    b3 = (3.+2.*r3)/12.
    assert len(y0)==len(dy0)
    ys = np.empty((1+nt,len(y0)))
    dys = np.empty((1+nt,len(dy0)))
    ys[0,:] = y0
    dys[0,:] = dy0
    for m in range(nt):
        g1 = ys[m]+c1*dt*dys[m]
        fg1 = ode_fun(g1,**args)
        g2 = ys[m]+dt*(c2*dys[m]+dt*a21*fg1)
        fg2 = ode_fun(g2,**args)
        g3 = ys[m]+dt*(c3*dys[m]+dt*a32*fg2)
        fg3 = ode_fun(g3,**args)
        ys[m+1] = ys[m]+dt*(dys[m]+dt*(bb1*fg1+bb2*fg2+bb3*fg3))
        dys[m+1] = dys[m]+ dt*(b1*fg1+b2*fg2+b3*fg3)
    return ys,dys
    
def ode_fun(y,C0=0.0,AR=1.,U=10.0,N=10):
    ez = C0+0.5*w(y,N,U)
    return np.array([-0.5*ez*dwdx(y,N,U),
                     -0.5*ez*dwdy(y,N,U)])

In [ ]:
# Setup parameters and initial conditions
U = 10.0
AR = 1.0
N = 10
dt = 0.5**7
nt = 2**(10+7)
y0 = [0.2,0.8]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)
C0 = -0.5*w(y0,N,U)+ez0
print(ode_fun(y0,C0=C0,AR=AR,U=U,N=N))

# Specify time stepping and solve
ts = np.linspace(0,nt*dt,nt+1)
ys,dys = RKN_method(ode_fun,y0,dy0,dt,nt,args={'C0':C0,'AR':AR,'U':U,'N':N})

# Plot the trajectory
plt.plot(ys[:,0],ys[:,1])
plt.plot([-AR,AR,AR,-AR,-AR],[-1,-1,1,1,-1],'k-')
plt.gca().set_aspect(1.0)
plt.show()

# Check the Hamiltonian
Ham = lambda y,dy:0.5*(dy[0]**2+dy[1]**2+(C0+0.5*w(y,N,U))**2) 
plt.plot(ts,Ham(ys.T,dys.T)-0.5)
plt.show()
if False:
    plt.plot(ts[:100],Ham(ys.T,dys.T)[:100]-0.5)
    plt.show()

In [ ]:
# For comparison, solve using scipy's solve_ivp
def ode_fun_first_order(t,y,C0=0.0,AR=1.,U=10.0,N=10):
    ez = C0+0.5*w(y,N,U)
    return np.array([y[2],y[3],-0.5*ez*dwdx(y,N,U),-0.5*ez*dwdy(y,N,U)])

# Check and solve
print(ode_fun_first_order(0,y0+dy0,C0=C0,AR=AR,U=U,N=N))
solution = solve_ivp(ode_fun_first_order,[ts[0],ts[-1]],y0+dy0,t_eval=ts,
                     method='DOP853',rtol=1.0E-12,atol=1.0E-9,
                     args=(C0,AR,U,N),)
ys = solution.y[:2].T
dys = solution.y[2:].T

# Plot the trajectory
plt.plot(ys[:,0],ys[:,1])
plt.plot([-AR,AR,AR,-AR,-AR],[-1,-1,1,1,-1],'k-')
plt.gca().set_aspect(1.0)
plt.show()

# Check the Hamiltonian
Ham = lambda y,dy:0.5*(dy[0]**2+dy[1]**2+(C0+0.5*w(y,N,U))**2) 
plt.plot(ts,Ham(ys.T,dys.T)-0.5)
plt.show()
if False:
    plt.plot(ts[:100],Ham(ys.T,dys.T)[:100]-0.5)
    plt.show()